In [10]:
import pandas as pd
import psycopg2
#to see progress bars
from tqdm.notebook import tqdm

# First task: Download the Files from the Birmingham City Council Website 

In order to download *all* de Excel files from the Birmingham City Council datasets website, I created a 3-column Excel file containing containing the following columns: 
- *Año*: the year of the report
- *Mes*: the month of the report
- *Link*: The URL from which I can download the Excel File. 

This is to automate loading the Excel Files, without downloading them into my computer. I named this file **file_sources.xls**:

In [92]:
sources_df = pd.read_excel('file_sources.xls',sep=';')

Now I will read the URLs, and store the content of them in a series of Pandas DataFrames from the Excel Files. I will hold them in a list of Pandas Data Frames called **files**. This procedure will take some time (I put a small progress bar). If any of the files has a problem (e.g. couldn't be downloaded or opened), this procedure will raise an exception. 

In [97]:
num_files = len(sources_df)
num_records = 0
files = []
for idx,source in tqdm(sources_df.iterrows(),
                       total=num_files,
                       desc='URLs downloaded'):
    mes = source['Mes']
    ano = source['Año']
    url_source = str(source['Link'])
    try:
        data_file = pd.read_excel(url_source)
        files.append(data_file)
        num_records += int(len(data_file))
    except Exception as e:
        print('Problem with report from', mes,'of the year',ano)
        print(e)
#at the end print the number of records found and the number of files obtained
print('Total number of records',num_records)
#update the number of files if any should fail
num_files = len(files)
print('Downloaded files',num_files)

URLs downloaded:   0%|          | 0/45 [00:00<?, ?it/s]

Problem with report from DEC of the year 2017
Unsupported format, or corrupt file: Expected BOF record; found b'REFERENC'
Total number of records 185909
Downloaded files 44


# Second task: Understanding the structure of the files 

According to the output we read succesfully 44 out of 45 files (only December 2017 was corrupt), and stored them as individual DataFrames. Moreover, we are dealing with a dataset of 185909 credit card payments distributed in montly reports starting in April 2014 all the way up to January 2018, with two exceptions: there is no record of transactions in April 2015, and the file for December 2017. 

Now, I wish to see the variables (columns) present in each file, and give myself an idea about what is contained in them. Moreover, I am interested in seeing if there are common variables in each one. I will store all variables (column names) found int the files in a set called **vars_found**. 

In [98]:
vars_found = set()
for file in tqdm(files):
    vars_found = vars_found.union(set(file.columns))
for var in vars_found:
    print(var)

  0%|          | 0/44 [00:00<?, ?it/s]

BILLING CUR CODE.1
ORIGINAL GROSS AMT
BILLING CUR CODE
Directorates
TRANS CAC DESC 1
Directorate
TRANS CAC CODE 3
Directorate 
TRANS CAC CODE 2
TRANS CAC CODE 1
BILLING GROSS AMT
MERCHANT NAME
TRANS DATE
ORIGINAL CUR
TRANS TAX AMT
CARD NUMBER
Unnamed: 10
TRANS CAC DESC 2
TRANS VAT DESC


Notice that since these variables were obtained by *adding* the variables found in each file, it is only natural that they show in at least one of the files. In order to detect which variables are *global* (i.e. present in each file) we will measure variables' *prevalence* among the files:

$$prevalence = \frac{\# \text{files in which it shows}}{\#\text{number of files}}\times 100\%$$

this prevalence will give us an idea of what *global variables* are. 

In [99]:
#create the prevalence dictionary as a dictionary by comprehension
var_prevalence = {var:0 for var in vars_found}
#now we traverse the list of DataFrames
for file in files:
    file_vars = list(file.columns)
    for var in file_vars:
            var_prevalence[var] +=1
#now we will create a variables dataframe
var_df = pd.DataFrame(columns=['var','total'])
var_df = var_df.from_dict(var_prevalence,orient='index')
#now we tailor the DataFrame
var_df = var_df.reset_index()
var_df = var_df.rename({'index':'var',0:'num_files'},axis=1)
var_df['prevalence'] = 100.0*var_df['num_files']/float(num_files)
#now we see the dataframe
var_df

,var,num_files,prevalence
0,BILLING CUR CODE.1,1,2.272727
1,ORIGINAL GROSS AMT,44,100.000000
2,BILLING CUR CODE,21,47.727273
3,Directorates,2,4.545455
4,TRANS CAC DESC 1,44,100.000000
5,Directorate,40,90.909091
6,TRANS CAC CODE 3,44,100.000000
7,Directorate,1,2.272727
8,TRANS CAC CODE 2,44,100.000000
9,TRANS CAC CODE 1,44,100.000000


Notice that we have multiple fields with similar names. For example, notice that the *Directorate* field comes with variations, occupying 43 out of the 44 files. Since the prevalence is not 100%, we will *discard* this variable for the moment. Now I wish to isolate all global variables (i.e. variables with a  100% prevalence), I will store these in a list called *global_vars*

In [100]:
#isolate global variables filtering by 100% prevalence
var_df.loc[var_df['prevalence'] == 100.0]
#store these variables in a list. 
global_vars = list(var_df.loc[var_df['prevalence'] == 100.0]['var'])

# Third Task: Create a single unified Dataset from Global Variables

Now that I have isolated these global variables, I wish to create a single DataFrame from it, so that I can perform analysis on it in another Notebook.

In [110]:
#create global dataset
global_dataset = pd.DataFrame(columns=global_vars)
#append the transactions from each monthly report
for file in tqdm(files):
    sub_data = file[global_vars]
    global_dataset = global_dataset.append(sub_data,ignore_index=True)
#I will create a unified transaction id using the index
global_dataset = global_dataset.reset_index()
#rename the former index as transac_id
global_dataset.rename({'index':'transac_id'})
print('number of records',len(global_dataset))
print('number of variables',len(global_vars))

  0%|          | 0/44 [00:00<?, ?it/s]

number of records 185909
number of variables 10


And now, we can check a little bit of this dataset, to get the feeling of what these global variables describe.

In [111]:
global_dataset.sample(10)

,index,ORIGINAL GROSS AMT,TRANS CAC DESC 1,TRANS CAC CODE 3,TRANS CAC CODE 2,TRANS CAC CODE 1,MERCHANT NAME,TRANS DATE,CARD NUMBER,TRANS CAC DESC 2,TRANS VAT DESC
66254,66254,6.20,Vehicle OthrunCosts,A00,RLBMC,K080,park mobile,2015-09-14,************2909,Technical Unit,VR
58201,58201,19.52,Supplies & Sev Mic,A00,REAEC,MC70,leaway s/stn,2015-07-15,************3866,Gossey Lane Junior & Infant,VR
100955,100955,23.98,Equip Operational,A00,RUAG1,L100,amazon uk marketplace,2016-05-27,************0407,Yardley Crematorium General,VZ
178904,178904,775.00,Promotions/Marketing,A00,RP005,L5H0,www.onlinecalendarsho,2017-11-13,************3475,City Centre,VZ
121669,121669,43.60,Purchases Food,A00,REAAB,L220,greggs - s0542,2016-10-19,************7065,The City of Birmingham School,VZ
158918,158918,6.22,Water Services,A00,RDP7G,J280,www.stwater.co.uk,2017-07-24,************3929,Gazzette Building 168 Corpn St,VZ
72120,72120,50.00,Equip Operational,A00,REAHH,L100,www.theworks.co.uk,2015-11-10,************9823,Moseley CE Junior & Infant,VZ
101274,101274,1184.90,Other Third Parties,A00,RJADC,N060,travelodge website,2016-06-17,************6316,Homeless Private Sector Accom,NaN
83199,83199,34.70,Equip Other,A00,REAJF,L120,amazon uk retail,2016-01-26,************7538,Paget Primary (NC),VZ
63287,63287,44.00,Equip Operational,A00,REAXZ,L100,tesco direct,2015-09-24,************3989,Hamilton,VZ


# Fourth Task: Data cleanup

From this small sample of the Dataset, we can infer that the variables stand for the following:

- **ORIGINAL GROSS AMT** the amount paid in the transaction
- **TRANS CAC DESC 1** A description of the transaction's purpose
- **MERCHANT NAME** The name of the store/facility that is the recipient of the transaction
- **TRANS DATE** the date on which the transaction occured
- **CARD NUMBER** the number of the card used to pay 
- **TRANS CAC DESC 2** A more detailed description of the store/facility where the transaction ocurred
- **TRANS CAC CODE 1,2,3** Internal codes to describe the transaction. 
- **TRANS VAT DESC** another code describing the transaction. 

Let's simplify the names and remove spaces

In [112]:
global_dataset.rename({'ORIGINAL GROSS AMT':'amount', 
                     'TRANS CAC DESC 1':'cac_desc_1', 
                     'TRANS CAC CODE 3':'cac_code_3',
                     'TRANS CAC CODE 2':'cac_code_2', 
                     'TRANS CAC CODE 1':'cac_code_1', 
                     'MERCHANT NAME':'merchant', 
                     'TRANS DATE':'date',
                     'CARD NUMBER':'card', 
                     'TRANS CAC DESC 2':'cac_desc_2', 
                     'TRANS VAT DESC':'vat_desc'},
                      axis=1,
                     inplace=True)
#Let's see how it looks. 
global_dataset.sample(5)

,index,amount,cac_desc_1,cac_code_3,cac_code_2,cac_code_1,merchant,date,card,cac_desc_2,vat_desc
136078,136078,40.24,Vehicle Fuel,A00,RTG15,K020,tesco pfs 4203,2017-02-01,************0479,Enforcement Team,VR
40329,40329,3.10,Vehicle OthrunCosts,A00,RV05X,K080,park mobile,2015-02-12,************8665,Admin Management Support,VZ
77427,77427,13.00,Equip Operational,A00,REACF,L100,amazon uk marketplace,2015-12-15,************2231,Cherry Orchard Junior & Infant,VZ
154988,154988,250.00,Bank & Goro ChgsS,A00,REAXL,L540,home bargains,2017-07-02,************5190,Baskerville,VR
72049,72049,69.80,Vehicle Fuel,A00,RV024,K020,morrisons petrol,2015-11-06,************3417,Leach Green Lane CH,VR


We see now that there is a combination of categorical and numerical variables including one date-time field. Let's see the types registered by Pandas to see if we need to cast any variable for analysis. 

In [114]:
global_dataset.dtypes

index                  int64
amount               float64
cac_desc_1            object
cac_code_3            object
cac_code_2            object
cac_code_1            object
merchant              object
date          datetime64[ns]
card                  object
cac_desc_2            object
vat_desc              object
dtype: object

Since the *card* field contains only the four last digits of the card, it doesn't make much sense to keep the trail of asterisks. Let's remove it using a lambda function

In [115]:
global_dataset['card'] = global_dataset['card'].apply(lambda x: str(x)[-4:])
global_dataset.head(3)

,index,amount,cac_desc_1,cac_code_3,cac_code_2,cac_code_1,merchant,date,card,cac_desc_2,vat_desc
0,0,52.32,Vehicle Fuel,A00,RV12N,K020,shell kings 587,2014-04-29,5770,African-Caribbean DC,VR
1,1,65.82,Vehicle Fuel,A00,RV12N,K020,shell kings 587,2014-04-04,5770,African-Caribbean DC,VR
2,2,41.35,Vehicle Fuel,A00,RV11Y,K020,tesco pfs 2484,2014-04-07,8738,"Marsh Lane Dce, 79, B23",VR


Now, I will only check for Nulls and Nans and remove them. In the analysis part I will determine if there are *strange* transactions upon checking expected values and whatnot. 

In [121]:
if global_dataset.isnull().values.any():
    print('There are null records')
    print('----------------------')
    num_null_records = global_dataset.isnull().sum().sum()
    print('There are', num_null_records)
    print(global_dataset.isnull().sum())

There are null records
----------------------
There are 10822
index            0
amount           1
cac_desc_1     105
cac_code_3     293
cac_code_2     236
cac_code_1     105
merchant         1
date             1
card             0
cac_desc_2     236
vat_desc      9844
dtype: int64


Notice that the most nulls are in the *vat_desc* column (almost 10 thousand), they are too many to simply drop. However, I will drop the records where the date or merchant are null, alongside with the records missing any CAC value (CAC code and cac desc). There is only one transaction missing it's value. I will drop it. 

In [123]:
global_dataset = global_dataset.dropna(subset=['cac_desc_1',
                                               'cac_desc_2',
                                               'cac_code_1',
                                               'cac_code_2',
                                               'cac_code_3',
                                               'date',
                                               'amount'])
print('Number of records after cleanup',len(global_dataset))

Number of records after cleanup 185480


After this cleanup we went from 185'909 records to 185'480 records. This is just a decrease in 429 records, that accounts for just 0.23% of the data. 

# Fifth Task: Save the Dataset to a CSV File. 

Now, we will save this dataset in a single csv file for analysis, that we will perform in another Jupyter Notebook. Since I am latino, I use the semi-colon as separator (so I can open this also in Excel), because I use the european number format (with a decimal comma, instead of a decimal point). I will mind this when opening the file in the next Notebook. 

In [124]:
global_dataset.to_csv('credit_card_transactions.csv',
                      sep=';', #I will use the semi colon
                      decimal=',', #I will use the european decimal comma instead of the decimal point
                      index=False) #I will drop the index from the DataFrame. 